In [4]:
import pandas as pd
cols = list(pd.read_csv("stage3.csv", nrows =1))
df= pd.read_csv("stage3.csv", usecols =[i for i in cols if i != 'sources'])
#df = pd.read_csv('/content/stage3.csv', error_bad_lines=False)
print(df)

          version https://git-lfs.github.com/spec/v1
0  oid sha256:c752945fb1d9252a61fbd2abdd8849999cc...
1                                     size 149698584
